In [843]:
import numpy as np
import pandas as pd
import re
import string
import pickle

from collections import OrderedDict

In [844]:
correct_bad_words_1 = pd.read_csv('badwords.txt', names=['wrong', 'right'])

In [845]:
correct_bad_words_2 =  pd.read_csv('correct_words.csv')

In [846]:
with_whitespace = correct_bad_words_2[correct_bad_words_2['wrong'].str.contains(' ')]

In [847]:
oov = pickle.load(open('oov', 'rb'))

In [848]:
bad_words_oov = correct_bad_words_2[correct_bad_words_2['wrong'].isin(oov)]

In [849]:
len(correct_bad_words_1), len(with_whitespace), len(bad_words_oov)

(549, 74, 44)

In [850]:
correct_bad_words = correct_bad_words_1.merge(with_whitespace, on=['wrong', 'right'], how='outer')
correct_bad_words = correct_bad_words.merge(bad_words_oov, on=['wrong', 'right'], how='outer')

In [851]:
bad_words = set(correct_bad_words[correct_bad_words['right'].isnull()]['wrong'].str.lower().tolist())

In [852]:
corrections = correct_bad_words[~correct_bad_words['right'].isnull()]

In [853]:
corrections['id'] = corrections['right'].factorize()[0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [854]:
word_corections = corrections[corrections['wrong'].str.isalpha()]

In [855]:
char_corections = corrections[~corrections['wrong'].str.isalpha()]

In [856]:
char_corrects_encode = dict(zip(char_corections.wrong.str.lower(), char_corections.id))
char_corrects_decode = dict(zip(char_corections.id, char_corections.right.str.lstrip()))

In [857]:
word_corrects = dict(zip(word_corections.wrong.str.lower(), word_corections.right.str.lstrip()))

In [858]:
char_corrects_encode = OrderedDict(sorted(char_corrects_encode.items(), key=lambda t: len(t[0]), reverse=True))

In [859]:
repl = {
    'macedonians': 'macedonian people',
    'unsalvageably': 'irrecoverably',
    'ukrainians': 'ukrainian people',
    'assyrians': 'assyrian people',
    'copyvios': 'copyvio',
    'yugoslav': 'yugoslavia',
    'deletionists': 'deletions',
    'meatpuppet': 'meat puppet',
    'meatpuppetry': 'meat puppet',
    'sockpuppet': 'sock puppet',
    'sockpuppeting': 'sock puppet',
    'herzegovina': 'bosnia',
    'azerbaijani': 'azerbaijan',
    'watchlisted': 'watchlist',
    'srebrenica': 'bosnia',
    'moldovan': 'moldova',
    'azeri': 'caucasian',
    'sockmaster': 'sock master',
    'sockpuppetter': 'sock master',
    'niggerpuppets': 'nigger puppets',
    'belarusian': 'ukrainian',
    'jewishness': 'cheapness',
    'vandalisim': 'vandalism',
    'vandolism': 'vandalism',
    'uzbek': 'uzbekistan people',
    'tajik': 'uzbekistan people',
    'wikinazi': 'wiki nazi',
    'deletin': 'deleting',
    'bulgars': 'bulgarians',
    'islams': 'islam people',
    'voluntaryslave': 'voluntary slave',
    'assyrians': 'syrians',
    'deleate': 'delete'
}

In [860]:
def split_text_and_digits(text, regexps):
    for regexp in regexps:
        result = regexp.match(text)
        if result is not None:
            return ' '.join(result.groups())
    return text

def _is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)


QUOTED_STRING_RE = re.compile(
    r"(?P<quote>\"\")(?P<string>.*?)(?<!\\)(?P=quote)")


def parse_quoted_string(search_string):
    r"""
    >>> s = '...And the gold of \'the knight\\\'s good banner\' Still waved...'
    >>> parse_quoted_string(s)
    "the knight\\'s good banner"
    >>> s = '"To save my lady!" Fast rode \'the knight\'... by "Stephen Crane"'
    >>> parse_quoted_string(s)
    'To save my lady!'
    >>> print(QUOTED_STRING_RE.findall(s))
    [('"', 'To save my lady!'), ("'", 'the knight'), ('"', 'Stephen Crane')]
    """
    match = QUOTED_STRING_RE.search(search_string)
    if match:
        if match.group('string').strip() in bad_words:
            search_string = re.sub(QUOTED_STRING_RE, 'bad word', search_string)
    return search_string


from nltk.tokenize import RegexpTokenizer, TweetTokenizer

tokenizer = RegexpTokenizer(r'\w+')
# tokenizer=TweetTokenizer()

def clean(comment):
    """
    This function receives comments and returns clean word-list
    """
    #Convert to lower case , so that Hi and hi are the same
    comment=comment.lower()
    #remove \n
    comment=re.sub(r"\n",". ",comment)
    comment=re.sub(r"\\n\n",". ",comment)
    
    #Special cases
    comment=re.sub(r"fucksex","fuck sex",comment)
    comment=re.sub(r"anti-semitism","antisemitism",comment)
    
    #Chinese bad word
    comment=re.sub(r"幹","fuck",comment)
    comment=re.sub(r"死","die",comment)
    comment=re.sub(r"他妈的","fuck",comment)
    comment=re.sub(r"去你妈的","fuck off",comment)
    comment=re.sub(r"肏你妈","fuck your mother",comment)
    comment=re.sub(r"肏你祖宗十八代","your ancestors to the 18th generation",comment)
    # remove leaky elements like ip,user
    comment=re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}","",comment)
    #removing usernames
#     comment=re.sub("\[\[.*\]","",comment)
    


    comment = re.sub(r"what's", "what is ", comment)
    comment = re.sub(r"\'s", " ", comment)
    comment = re.sub(r"\'ve", " have ", comment)
    comment = re.sub(r"can't", "cannot ", comment)
    comment = re.sub(r"n't", " not ", comment)
    comment = re.sub(r"i'm", "i am ", comment)
    comment = re.sub(r"\'re", " are ", comment)
    comment = re.sub(r"\'d", " would ", comment)
    comment = re.sub(r"\'ll", " will ", comment)
#     comment = re.sub(r"wtf","what the fuck", comment)    
    comment = re.sub(r"I", "one", comment)
    comment = re.sub(r"II", "two", comment)
    comment = re.sub(r"III", "three", comment)
    comment = re.sub(r'牛', "cow", comment)
    comment=re.sub(r"mothjer","mother",comment)
    comment=re.sub(r"g e t  r i d  o f  a l l  i  d i d  p l e a s e  j a ck a s s",
                   "get rid of all i did please jackass",comment)
    comment=re.sub(r"withought","with out",comment)
    
    
    #Replace LINKS
    comment = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', comment)
    
#     comment=substitute_repeats(comment)
    comment = re.sub(r'(.)\1{2,}', r'\1', comment)

    
    #Bad chars correction
    comment = ' ' +  comment
    for wrong, right in char_corrects_encode.items():
        comment = comment.replace(' ' + wrong,  ' __'+ str(right) + '__ ')
#     print(comment)
    for wrong, right in char_corrects_decode.items():
        comment = comment.replace( '__'+ str(wrong) + '__', ' ' + right + ' ')
#     print(comment)
#     comment = parse_quoted_string(comment)
     
    comment = comment.replace('&', ' and ')
    comment = comment.replace('@', ' at ')
    comment = comment.replace('雲水','')
    
    #Tokenize
    words=tokenizer.tokenize(comment)   
    regexps = [re.compile("([a-zA-Z]+)([0-9]+)"), re.compile("([0-9]+)([a-zA-Z]+)")]
    words = [split_text_and_digits(token, regexps) for token in words]
    words = ' '.join(words).split()
    #Bad words correction
    words=[word_corrects[word] if word in word_corrects else word for word in words]
    words=[repl[word] if word in repl else word for word in words]
    words = ' '.join(words).split()
    
    words = list(filter(_is_valid_word, words))
    

    return " ".join(words)

In [861]:
text = """
does any1 think its dangerous to b on facebook	
"""

In [862]:
train = pd.read_csv('data/train.csv').fillna(' ')
test = pd.read_csv('data/test.csv').fillna(' ')

In [863]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [864]:
clean("damn 'preceeding unsigned.....yada...yada' What a")

'damn preceeding unsigned yada yada what a'

In [865]:
text = """
stfu u deleted our page. tool.	
"""

In [866]:
clean_text(text)

'stfu u deleted our page tool'

In [867]:
clean(text)

'stfu u deleted our page tool'

In [485]:
train['comment_text'].sample(1).iloc[0]

'"\nHey folks!  Let\'s watch throwing around the ""r"" word so loosely .... that constitutes a personal attack, and violates WP:CIVIL.   "'

In [868]:
%%time
test['prep_text'] = test['comment_text'].map(clean)

CPU times: user 1min 31s, sys: 8 ms, total: 1min 31s
Wall time: 1min 31s


In [869]:
%%time
train['prep_text'] = train['comment_text'].map(clean)

CPU times: user 1min 39s, sys: 152 ms, total: 1min 39s
Wall time: 1min 39s


In [870]:
test.to_csv('data/test_prep.csv', index=False)
train.to_csv('data/train_prep.csv', index=False)